In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('./data/train.csv')
df = df[df.store_nbr == 25].copy()
df = df.groupby(["date"])['unit_sales'].sum().reset_index()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')

FileNotFoundError: [Errno 2] No such file or directory: './data/train.csv'

In [ ]:
df.plot(x = 'date', y = 'unit_sales',  figsize=(15, 10))
plt.show()

In [ ]:
timeDay = pd.date_range(df['date'].min(), end=df['date'].max())
union = pd.DataFrame()
union["date"] = timeDay
union = pd.merge(left=union, right=df[["date", 'unit_sales']], how="left",on=["date"])
union["unit_sales"].fillna(0, inplace=True)

In [ ]:
union = union.set_index(pd.DatetimeIndex(union['date']))
union.drop(['date'], axis = 1, inplace = True)

In [ ]:
decomposition = seasonal_decompose(union, model='additive')
trend_part = decomposition.trend
seasonal_part = decomposition.seasonal
residual_part = decomposition.resid

In [ ]:
trend_part.plot(x = 'date', y = 'unit_sales',  figsize=(15, 10))
plt.show()

In [ ]:
trend_part = trend_part.dropna()
print("Размер трендовой составляющей " + str(trend_part.shape[0]))

In [ ]:
seasonal_part.plot(x = 'date', y = 'unit_sales',  figsize=(15, 10))
plt.show()

In [ ]:
seasonal_part = seasonal_part.dropna()
print("Размер сезональной составляющей " + str(seasonal_part.shape[0]))

In [ ]:
residual_part.plot(x = 'date', y = 'unit_sales',  figsize=(15, 10))
plt.show()

In [ ]:
trend_part = trend_part.dropna()
print("Размер случаной составляющей " + str(trend_part.shape[0]))

In [ ]:
def adfulleftDef(df):
    test = adfuller(df["unit_sales"])
    print('adf: ', test[0])
    print('p-value: ', test[1])
    print('Critical values: ', test[4])
    if test[0] > test[4]['5%']:  # проверка, больше ли критического полученное значение для нашего ряда
        print('ряд не стационарен')
    else:
        print('ряд стационарен')
adfulleftDef(union)

#  Oконные функции

In [ ]:
def Upper(value):
   return value.mean() + 3 * value.std()
def Lower(value):
   return value.mean() - 3 * value.std()

In [ ]:
union['sales_mean'] = union["unit_sales"].rolling(window=5).mean()
union['sales_std'] = union["unit_sales"].rolling(window=5).std()

union['upper']  = union["unit_sales"].rolling(window=30).apply(Upper, raw=False)
union['lower']  = union["unit_sales"].rolling(window=30).apply(Lower, raw=False)

union['date'] = union.index

In [ ]:
## визуализируем
f, ax = plt.subplots(1, 1, figsize=(18, 8))
ax.plot(union["date"], union["unit_sales"], c='black') ## исходные данные
ax.plot(union["date"], union["sales_mean"], c='slateblue') ## среднее данные
ax.plot(union["date"], union["sales_std"], c='salmon') ##  квадратическое отклонение 

ax.plot(union["date"], union["lower"], c='lime') ## нижняя граница Боллинджера 
ax.plot(union["date"], union["upper"], c='darkgreen') ## вверхняя граница Боллинджера 


ax.legend(['unit_sales', 'sales_mean', 'sales_std', "lower", 'upper'])
ax.grid()

In [ ]:
# Постройте оконное среднее с окном в 10 точек.
# Постройте экспоненциально среднее с окном в 7 точек (span=7).
N = 10
union['sales_mean '  + str(N)] = union["unit_sales"].rolling(window=10).mean()
N = 7
union['sales_ewn '  + str(N)] = union["unit_sales"].ewm(min_periods=N, span=N).mean()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(18, 8))
ax.plot(union["date"], union["sales_mean 10"], c='black') ## исходные данные
ax.plot(union["date"], union["sales_ewn 7"], c='slateblue') ## среднее данные


ax.legend(['unit_sales', 'sales_mean', 'sales_std', "lower", 'upper'])
ax.grid()

In [ ]:
#Напишите свою скользящую функцию, которая будет являться средним арифметическим между максимальным
# и минимальным значением unit_sales, используя окно 10.
def MeanMaxMin(value):
    return (value.max() + value.min()) / 2
union['sales_func'] = union["unit_sales"].rolling(window=10).apply(MeanMaxMin, raw=False)

In [ ]:
#Постройте скользящее среднее с окном 50 и экспоненциальное скользящее среднее с окном 10 (span=10).
#Укажите индексы датафрейма, на которых одно скользящее пересекается с другим
union['sales_mean_50'] = union["unit_sales"].rolling(window=50).mean()
union['sales_ewn_10'] = union["unit_sales"].ewm(min_periods=10, span=10).mean()

union['diff'] = union['sales_mean_50'] - union['sales_ewn_10']
union['diff'] = np.sign(union['diff']).diff()
union[union['diff'] != 0]['date']

In [ ]:
# Получите лаговые факторы с 7-го по 10-й предыдущих дня (лаг7, лаг8, лаг9, лаг10) и отобразите на графике
union = union[['date', 'unit_sales']]

for i in range(7,11):
    union['sale_{}'.format(i)] = union.unit_sales.shift(i)
    
## визуализируем
f, ax = plt.subplots(1, 1, figsize=(18, 8))
ax.plot(union.date, union.unit_sales) ## исходные данные
ax.plot(union.date, union['sale_' + str(7)],  c='g') 
ax.plot(union.date, union['sale_' + str(8)],  c='r') 
ax.plot(union.date, union['sale_' + str(9)],  c='b') 
ax.plot(union.date, union['sale_' + str(10)],  c='y') 
ax.legend(['unit_sales', 'sale_7', 'sale_8', 'sale_9', 'sale_10'])
ax.grid()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(18, 8))
ax.plot(union.date[:100], union.unit_sales[:100]) ## исходные данные
ax.plot(union.date[:100], union['sale_' + str(7)][:100],  c='g') 
ax.plot(union.date[:100], union['sale_' + str(8)][:100],  c='r') 
ax.plot(union.date[:100], union['sale_' + str(9)][:100],  c='b') 
ax.plot(union.date[:100],  union['sale_' + str(10)][:100],  c='y') 
ax.legend(['unit_sales','sale_7', 'sale_8', 'sale_9', 'sale_10'])
ax.grid()